In [1]:
# ----- 2D FSI code demo

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix, diags
from scipy.sparse.linalg import spsolve

In [21]:
# 配置文件处理器
# ============================
class ConfigHandler:
    def __init__(self, filename="control.dat"):
        self.params = {}
        with open(filename) as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    self.params[key.strip()] = eval(value.split('#')[0].strip())

    def get(self, key, default=None):
        return self.params.get(key, default)

In [22]:
# 非均匀网格生成器
# ============================
class GridGenerator:
    @staticmethod
    def generate_stretched_grid(nx, ny, Lx, Ly, stretch_ratio=1.1):
        xi = np.linspace(0, 1, nx)
        x = Lx * (np.exp(stretch_ratio * xi) - 1) / (np.exp(stretch_ratio) - 1)
        y = np.linspace(0, Ly, ny)
        return x, y

In [23]:
# 网格类
# ============================
class Grid:
    def __init__(self, config):
        self.config = config
        self.nx = config.get('nx', 64)
        self.ny = config.get('ny', 32)
        if config.get('grid_file', 0) == 1:
            self.load_from_file()
        else:
            self.generate_new_grid()
        self.initialize_mesh_metrics()

    def load_from_file(self):
        try:
            data = np.load('grid.dat', allow_pickle=True).item()
            self.xu, self.yu = data['xu'], data['yu']
            self.xv, self.yv = data['xv'], data['yv']
            self.xp, self.yp = data['xp'], data['yp']
        except FileNotFoundError:
            print("Grid file not found, generating new grid")
            self.generate_new_grid()

    def generate_new_grid(self):
        Lx = self.config.get('Lx', 4.0)
        Ly = self.config.get('Ly', 2.0)
        self.xu, self.yu = GridGenerator.generate_stretched_grid(self.nx, self.ny, Lx, Ly)
        self.xv, self.yv = self.xu.copy(), self.yu.copy()
        self.xp = 0.5 * (self.xu[1:] + self.xu[:-1])
        self.yp = 0.5 * (self.yu[1:] + self.yu[:-1])
        if self.config.get('grid_file', 0) == 1:
            self.save_grid()

    def save_grid(self):
        grid_data = {
            'xu': self.xu, 'yu': self.yu,
            'xv': self.xv, 'yv': self.yv,
            'xp': self.xp, 'yp': self.yp
        }
        np.save('grid.dat', grid_data)

    def initialize_mesh_metrics(self):
        self.dxu = np.diff(self.xu)
        self.dyu = np.diff(self.yu)
        self.dxv = np.diff(self.xv)
        self.dyv = np.diff(self.yv)
        self.vol = np.outer(self.dyu, self.dxu)

In [24]:
# 流场类
# ============================
class FlowField:
    def __init__(self, grid, config):
        self.config = config
        self.grid = grid
        self.rho = config.get('rho', 1.0)
        self.nu_artificial = config.get('nu_artificial', 0.01)

        self.u = np.zeros((len(grid.yu), len(grid.xu))) + config.get('U_inlet', 1.0)
        self.v = np.zeros((len(grid.yv), len(grid.xv)))
        self.p = np.zeros((len(grid.yp), len(grid.xp)))

    def apply_boundary_conditions(self):
        self.u[:, 0] = self.config.get('U_inlet', 1.0)
        self.v[:, 0] = 0.0
        self.u[:, -1] = self.u[:, -2]
        self.v[:, -1] = self.v[:, -2]
        self.p[:, -1] = 0.0
        self.v[0, :] = 0.0
        self.v[-1, :] = 0.0

In [33]:
# 浸入边界类
# ============================
class ImmersedBoundary:
    def __init__(self, config, grid):
        length = config.get('ib_length', 0.5)
        angle = np.deg2rad(config.get('ib_angle', 30))
        center = config.get('ib_center', [1.0, 1.0])
        
        n_points = config.get('ib_points', 20)
        s = np.linspace(-length/2, length/2, n_points)
        self.points = np.array([
            [center[0] + si*np.cos(angle), 
             center[1] + si*np.sin(angle)] 
            for si in s
        ])
        self.grid = grid
    
    def delta_function(self, r, h):
        """4点离散delta函数"""
        r_abs = np.abs(r/h)
        if r_abs <= 1:
            return (1 + np.cos(np.pi*r_abs)) / (4*h)
        return 0.0
    
    def interpolate_velocity(self, Xk, u, v):
        """速度插值到浸没边界点"""
        # 找到最近的网格索引
        i = np.searchsorted(self.grid.xu, Xk[0]) - 1
        j = np.searchsorted(self.grid.yu, Xk[1]) - 1
        
        # 双线性插值
        x_ratio = (Xk[0] - self.grid.xu[i]) / self.grid.dxu[i]
        y_ratio = (Xk[1] - self.grid.yu[j]) / self.grid.dyu[j]
        
        u_interp = (1-x_ratio)*(1-y_ratio)*u[j,i] + \
                   x_ratio*(1-y_ratio)*u[j,i+1] + \
                   (1-x_ratio)*y_ratio*u[j+1,i] + \
                   x_ratio*y_ratio*u[j+1,i+1]
        
        v_interp = (1-x_ratio)*(1-y_ratio)*v[j,i] + \
                   x_ratio*(1-y_ratio)*v[j,i+1] + \
                   (1-x_ratio)*y_ratio*v[j+1,i] + \
                   x_ratio*y_ratio*v[j+1,i+1]
        
        return np.array([u_interp, v_interp])
    
    def spread_force(self, Xk, Fk, fx, fy):
        for i in range(len(self.grid.xu)):
            if i >= len(self.grid.dxu):
                dx = self.grid.xu[i] - Xk[0]
                dxu_val = self.grid.dxu[-1]  # 取边界值
            else:
                dx = self.grid.xu[i] - Xk[0]
                dxu_val = self.grid.dxu[i]

            for j in range(len(self.grid.yu)):
                if j >= len(self.grid.dyu):
                    dy = self.grid.yu[j] - Xk[1]
                    dyu_val = self.grid.dyu[-1]
                else:
                    dy = self.grid.yu[j] - Xk[1]
                    dyu_val = self.grid.dyu[j]

                delta = self.delta_function(dx, dxu_val) * self.delta_function(dy, dyu_val)
                fx[j, i] += Fk[0] * delta * dxu_val * dyu_val
                fy[j, i] += Fk[1] * delta * dxu_val * dyu_val
        return fx, fy


In [26]:
# 多网格求解器类
# ============================
class MultiGridSolver:
    def __init__(self, grid, levels=3):
        self.grid = grid
        self.levels = levels
        self.initialize_operators()

    def initialize_operators(self):
        self.A_levels = []
        self.R_levels = []
        self.P_levels = []

        for l in range(self.levels):
            nx = self.grid.nx // (2**l)
            ny = self.grid.ny // (2**l)
            self.A_levels.append(self.build_poisson_matrix(nx, ny))

            if l < self.levels - 1:
                nxc = nx // 2
                nyc = ny // 2
                self.R_levels.append(self.build_identity(nyc, nxc, nx, ny))
                self.P_levels.append(self.build_identity(nx, ny, nxc, nyc))

    def build_identity(self, nx_to, ny_to, nx_from, ny_from):
        N_from = nx_from * ny_from
        N_to = nx_to * ny_to
        I = lil_matrix((N_to, N_from))
        for i in range(min(N_to, N_from)):
            I[i, i] = 1.0
        return I.tocsr()

    def V_cycle(self, b, x0, level=0):
        if level == self.levels - 1:
            return spsolve(self.A_levels[level], b)

        x = self.gauss_seidel(self.A_levels[level], b, x0, iterations=2)
        residual = b - self.A_levels[level].dot(x)
        b_coarse = self.R_levels[level].dot(residual)
        e_coarse = self.V_cycle(b_coarse, np.zeros_like(b_coarse), level + 1)
        x += self.P_levels[level].dot(e_coarse)
        x = self.gauss_seidel(self.A_levels[level], b, x, iterations=2)
        return x

    def build_poisson_matrix(self, nx, ny):
        A = lil_matrix((nx * ny, nx * ny))
        for j in range(ny):
            for i in range(nx):
                row = j * nx + i
                A[row, row] = -4.0
                if i > 0: A[row, row - 1] = 1.0
                if i < nx - 1: A[row, row + 1] = 1.0
                if j > 0: A[row, row - nx] = 1.0
                if j < ny - 1: A[row, row + nx] = 1.0
        return A.tocsr()

    def gauss_seidel(self, A, b, x, iterations):
        for _ in range(iterations):
            for i in range(A.shape[0]):
                row_start = A.indptr[i]
                row_end = A.indptr[i + 1]
                Ax = sum(A.data[row_start:row_end] * x[A.indices[row_start:row_end]])
                x[i] = (b[i] - (Ax - A[i, i] * x[i])) / A[i, i]
        return x

In [36]:
# FSI求解器主类
# ============================
class FSISolver:
    def __init__(self, config):
        self.config = config
        self.grid = Grid(config)
        self.field = FlowField(self.grid, config)
        self.ib = ImmersedBoundary(config, self.grid)
        self.mg_solver = MultiGridSolver(self.grid, config.get('mg_levels', 3))
        
        self.dt = config.get('dt', 0.005)
        self.steps = config.get('steps', 100)
    
    def second_order_upwind(self, phi, idx, delta, direction):
        if direction > 0:
            if idx < 2:
            # 使用一阶迎风或中心差分替代
                return (phi[idx] - phi[idx-1]) / delta if idx > 0 else 0.0
            return (3*phi[idx] - 4*phi[idx-1] + phi[idx-2]) / (2*delta)
        else:
            if idx > len(phi) - 3:
            # 使用一阶迎风或中心差分替代
                return (phi[idx+1] - phi[idx]) / delta if idx < len(phi) - 1 else 0.0
            return (-phi[idx+2] + 4*phi[idx+1] - 3*phi[idx]) / (2*delta)

    
    def solve_momentum(self):
        """求解动量方程（含人工粘性）"""
        nu = self.field.nu_artificial
        u, v = self.field.u, self.field.v
        
        u_star = u.copy()
        v_star = v.copy()
        
        for j in range(1, self.grid.ny-1):
            for i in range(1, self.grid.nx-1):
                # 对流项（二阶迎风）
                du_dx = self.second_order_upwind(u[j,:], i, 
                    self.grid.dxu[i], u[j,i])
                du_dy = self.second_order_upwind(u[:,i], j,
                    self.grid.dyu[j], v[j,i])
                
                # 扩散项（人工粘性）
                diffusion = nu * (
                    (u[j,i+1] - 2*u[j,i] + u[j,i-1])/self.grid.dxu[i]**2 +
                    (u[j+1,i] - 2*u[j,i] + u[j-1,i])/self.grid.dyu[j]**2
                )
                
                u_star[j,i] = u[j,i] + self.dt * (
                    -u[j,i]*du_dx - v[j,i]*du_dy + diffusion
                )
        
        return u_star, v_star
    
    def solve_pressure(self, u_star, v_star):
        # 构建源项
        div_u = np.zeros_like(self.field.p)
        for j in range(1, self.grid.ny-1):
            for i in range(1, self.grid.nx-1):
                div_u[j,i] = (u_star[j,i+1] - u_star[j,i-1])/(2*self.grid.dxu[i]) + \
                             (v_star[j+1,i] - v_star[j-1,i])/(2*self.grid.dyu[j])
        
        # 调用多网格求解器
        p_corr = self.mg_solver.V_cycle(
            div_u.flatten()/self.dt,
            self.field.p.flatten()
        ).reshape(self.field.p.shape)
        
        return p_corr
    
    def correct_velocity(self, u_star, v_star, p_corr):
        """速度修正"""
        u = u_star.copy()
        v = v_star.copy()
        
        for j in range(1, self.grid.ny-1):
            for i in range(1, self.grid.nx-1):
                dpdx = (p_corr[j,i+1] - p_corr[j,i-1])/(2*self.grid.dxu[i])
                dpdy = (p_corr[j+1,i] - p_corr[j-1,i])/(2*self.grid.dyu[j])
                
                u[j,i] -= self.dt/self.field.rho * dpdx
                v[j,i] -= self.dt/self.field.rho * dpdy
        
        return u, v
    
    def run(self):
        """主求解循环"""
        for step in range(self.steps):
            print(f"Step {step+1}/{self.steps}")
            
            # 1. 计算浸没边界力
            fx = np.zeros_like(self.field.u)
            fy = np.zeros_like(self.field.v)
            for Xk in self.ib.points:
                Uk = self.ib.interpolate_velocity(Xk, self.field.u, self.field.v)
                Fk = (np.zeros(2) - Uk) * self.field.rho / self.dt
                fx, fy = self.ib.spread_force(Xk, Fk, fx, fy)
            
            # 2. 求解动量方程
            u_star, v_star = self.solve_momentum()
            
            # 3. 求解压力
            p_corr = self.solve_pressure(u_star, v_star)
            
            # 4. 修正速度压力场
            self.field.u, self.field.v = self.correct_velocity(u_star, v_star, p_corr)
            self.field.p += p_corr
            
            # 5. 应用边界条件
            self.field.apply_boundary_conditions()
    
    def visualize(self):
        """结果可视化"""
        plt.figure(figsize=(12,6))
        plt.contourf(self.grid.xp, self.grid.yp, self.field.p, levels=20, cmap='RdBu')
        plt.colorbar(label='Pressure')
        plt.quiver(self.grid.xu, self.grid.yu, self.field.u, self.field.v, scale=30)
        plt.scatter(self.ib.points[:,0], self.ib.points[:,1], c='red', s=20)
        plt.title('FSI Simulation with Immersed Boundary')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()

In [37]:

# 主程序
# ============================
if __name__ == "__main__":
    config = ConfigHandler("control.dat")
    solver = FSISolver(config)
    solver.run()
    solver.visualize()

Step 1/100


IndexError: index 1953 is out of bounds for axis 0 with size 1953